### Install & Import Required Libraries

In [2]:
#!pip install snowflake-connector-python pandas snowflake-sqlalchemy mlxtend

In [1]:
import pandas as pd
import snowflake.connector
from getpass import getpass
from sqlalchemy import create_engine
from math import pi
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from mlxtend.frequent_patterns import apriori, association_rules
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

### Connect to Snowflake Account and fetch data

In [4]:
user = 'LiMi686'
password = getpass.getpass('Enter your Snowflake password: ')
account = 'TVRZIHA-IFB54421'
warehouse = 'COMPUTE_WH'
database = 'OLIST'
schema = 'PUBLIC'

# Create SQLAlchemy engine
engine = create_engine(
    f'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'
)

# Fetch data from tables
def fetch_table(table_name):
    query = f'SELECT * FROM {table_name};'
    with engine.connect() as conn:
        return pd.read_sql(query, conn)

# Load all required tables
orders_df = fetch_table('olist_orders')
customers_df = fetch_table('olist_order_customer')
geolocation_df = fetch_table('olist_geolocation')
products_df = fetch_table('olist_products')
sellers_df = fetch_table('olist_sellers')
order_items_df = fetch_table('olist_order_items')
payments_df = fetch_table('olist_order_payments')
reviews_df = fetch_table('olist_order_reviews')



Enter your Snowflake password:  ········


### Data Cleaning

In [6]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [ ]:
def clean_df(df, name):
    print(f"Before cleaning: {name} -> {df.shape}")
    df = df.drop_duplicates()
    df = df.dropna()
    print(f"After cleaning: {name} -> {df.shape}")
    return df

orders_df = clean_df(orders_df, "orders")
customers_df = clean_df(customers_df, "customers")
geolocation_df = clean_df(geolocation_df, "geolocation")
products_df = clean_df(products_df, "products")
sellers_df = clean_df(sellers_df, "sellers")
order_items_df = clean_df(order_items_df, "order_items")
payments_df = clean_df(payments_df, "payments")
reviews_df = clean_df(reviews_df, "reviews")

### Combine all tables and 

In [ ]:
# Merge 1: Orders + Customers
orders_customers = pd.merge(orders_df, customers_df, on='customer_id', how='inner')

# Merge 2: + Geolocation (via zip prefix)
orders_customers_geo = pd.merge(
    orders_customers,
    geolocation_df,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)

# Merge 3: + Order Items
orders_full = pd.merge(orders_customers_geo, order_items_df, on='order_id', how='inner')

# Merge 4: + Products
orders_full = pd.merge(orders_full, products_df, on='product_id', how='left')

# Merge 5: + Sellers
orders_full = pd.merge(orders_full, sellers_df, on='seller_id', how='left')

# Merge 6: + Payments
orders_full = pd.merge(orders_full, payments_df, on='order_id', how='left')

# Merge 7: + Reviews
orders_full = pd.merge(orders_full, reviews_df, on='order_id', how='left')

# Result
print(f"Unified dataset shape: {orders_full.shape}")
orders_full.head()


### Feature Engineering

delivery_time: Derived as the difference between delivery and purchase timestamps to quantify shipping speed.

avg_order_value: Aggregated total order price per customer to measure spending behavior.

Calculated total_orders per customer: To understand customer engagement.

Computed avg_review_score per customer: To capture customer satisfaction.

Extracted most_used_payment_type per customer: To profile payment preferences.

Derived top_product_categories per customer: To understand product interests.

In [ ]:
# Convert dates to datetime
date_cols = [
    'order_purchase_timestamp', 'order_approved_at', 
    'order_delivered_carrier_date', 'order_delivered_customer_date', 
    'order_estimated_delivery_date'
]

for col in date_cols:
    orders_full[col] = pd.to_datetime(orders_full[col])

# Feature 1: Delivery Time
orders_full['delivery_time'] = (orders_full['order_delivered_customer_date'] - orders_full['order_purchase_timestamp']).dt.days

# Feature 2: Estimated Delay (output is positive if late)
orders_full['estimated_delay'] = (orders_full['order_delivered_customer_date'] - orders_full['order_estimated_delivery_date']).dt.days

# Feature 3: Approval Delay
orders_full['approval_delay'] = (orders_full['order_approved_at'] - orders_full['order_purchase_timestamp']).dt.total_seconds() / 3600  # (time in hours)

# Feature 4: Freight-to-Price Ratio
orders_full['freight_ratio'] = orders_full['freight_value'] / orders_full['price']

# Clean infinite/NaN ratios
orders_full['freight_ratio'] = orders_full['freight_ratio'].replace([float('inf'), -float('inf')], pd.NA)
orders_full.dropna(subset=['freight_ratio'], inplace=True)

# Preview
orders_full[['delivery_time', 'estimated_delay', 'approval_delay', 'freight_ratio']].describe()


In [ ]:
# Group by customer_id
customer_features = orders_full.groupby('customer_id').agg({
    'order_id': 'nunique',
    'price': 'mean',
    'payment_type': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'review_score': 'mean',
    'customer_state': 'first'
}).reset_index()

customer_features.columns = [
    'customer_id', 
    'num_orders', 
    'avg_order_value', 
    'most_used_payment_type', 
    'avg_review_score', 
    'customer_state'
]

customer_features.head()


In [ ]:
order_level_features = orders_full.groupby('order_id').agg({
    'customer_id': 'first',
    'product_id': 'nunique',
    'price': 'sum',
    'freight_value': 'sum',
    'order_item_id': 'count',
    'delivery_time': 'mean',
    'estimated_delay': 'mean'
}).reset_index()

order_level_features.columns = [
    'order_id', 'customer_id', 'unique_products', 'total_order_value',
    'total_freight', 'num_items', 'avg_delivery_time', 'avg_estimated_delay'
]

order_level_features.head()


In [ ]:
# Filter only required columns
basket_df = orders_full[['order_id', 'product_category_name']].dropna()

# Create binary matrix: 1 if category present in order
basket_matrix = pd.crosstab(basket_df['order_id'], basket_df['product_category_name'])

basket_matrix = basket_matrix.map(lambda x: 1 if x > 0 else 0)

basket_matrix = basket_matrix.astype(bool)

basket_matrix.head()


### Encoding for Clustering

In [ ]:

# Combine customer and order-level features
clustering_data = pd.merge(order_level_features, customer_features, on='customer_id', how='left')

# Drop IDs
clustering_data_clean = clustering_data.drop(columns=['order_id', 'customer_id'])

# Define columns
num_cols = ['num_orders', 'avg_order_value', 'avg_review_score', 'unique_products', 'total_order_value',
            'total_freight', 'num_items', 'avg_delivery_time', 'avg_estimated_delay']
cat_cols = ['most_used_payment_type', 'customer_state']

# Build pipeline
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_cols)
])

clustering_prepared = preprocessor.fit_transform(clustering_data_clean)

print(f"Final shape for clustering: {clustering_prepared.shape}")


### Features - Visualizations

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(orders_full['review_score'].dropna(), bins=5, kde=True)
plt.title('Distribution of Review Scores')
plt.xlabel('Review Score')
plt.ylabel('Count')
plt.grid(True)
plt.show()


In [ ]:
avg_order_value = orders_full.groupby('customer_id')['price'].mean()

plt.figure(figsize=(6, 4))
sns.histplot(avg_order_value, bins=30, kde=True)
plt.title('Average Order Value per Customer')
plt.xlabel('Avg. Order Value')
plt.ylabel('Count')
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(data=orders_full, x='payment_type', order=orders_full['payment_type'].value_counts().index)
plt.title('Payment Method Distribution')
plt.xlabel('Payment Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


In [ ]:
#Time - based
orders_full['delivery_time'] = (orders_full['order_delivered_customer_date'] - orders_full['order_purchase_timestamp']).dt.days

plt.figure(figsize=(6, 4))
sns.histplot(orders_full['delivery_time'].dropna(), bins=30)
plt.title('Delivery Time (Days)')
plt.xlabel('Days to Deliver')
plt.ylabel('Number of Orders')
plt.grid(True)
plt.show()


In [ ]:
top_cats = orders_full['product_category_name'].value_counts().head(10)

plt.figure(figsize=(8, 5))
sns.barplot(x=top_cats.values, y=top_cats.index, palette='viridis', hue=top_cats.index, legend=False)
plt.title('Top 10 Product Categories')
plt.xlabel('Number of Purchases')
plt.ylabel('Product Category')
plt.show()


#### Time-based

In [ ]:
# Histogram of estimated_delay
sns.histplot(orders_full['estimated_delay'], bins=30)
plt.title('Estimated Delay Distribution')
plt.xlabel('Days (Positive = Late)')
plt.ylabel('Frequency')
plt.show()

# Boxplot of approval_delay by payment_type
sns.boxplot(x='payment_type', y='approval_delay', data=orders_full)
plt.title('Approval Delay by Payment Type')
plt.xlabel('Payment Type')
plt.ylabel('Approval Delay (Hours)')
plt.xticks(rotation=45)
plt.show()


#### Value-based

In [ ]:
# Scatter plot of freight_ratio vs. price
sns.scatterplot(x='price', y='freight_ratio', data=orders_full)
plt.title('Freight-to-Price Ratio vs Price')
plt.xlabel('Price')
plt.ylabel('Freight Ratio')
plt.show()

# Boxplot of freight_ratio by product_category_name (top 10 categories)
top_categories = orders_full['product_category_name'].value_counts().nlargest(10).index
sns.boxplot(data=orders_full[orders_full['product_category_name'].isin(top_categories)],
            x='product_category_name', y='freight_ratio')
plt.title('Freight Ratio by Product Category (Top 10)')
plt.xlabel('Product Category')
plt.ylabel('Freight Ratio')
plt.xticks(rotation=45)
plt.show()


#### Order level

In [ ]:
# Histogram of unique_products
sns.histplot(order_level_features['unique_products'], bins=20)
plt.title('Unique Products per Order')
plt.xlabel('Number of Unique Products')
plt.ylabel('Frequency')
plt.show()

# Bar plot of avg_estimated_delay per product category (top 10)
avg_delay = orders_full.groupby('product_category_name')['estimated_delay'].mean().reset_index()
top_delay = avg_delay.sort_values(by='estimated_delay', ascending=False).head(10)
sns.barplot(x='estimated_delay', y='product_category_name', data=top_delay)
plt.title('Avg Estimated Delay by Product Category (Top 10)')
plt.xlabel('Avg Estimated Delay (Days)')
plt.ylabel('Product Category')
plt.show()


#### Customer-level

In [ ]:
# Bar plot of num_orders per customer_state
orders_per_state = customer_features.groupby('customer_state')['num_orders'].sum().reset_index()
sns.barplot(x='customer_state', y='num_orders', data=orders_per_state)
plt.title('Number of Orders by State')
plt.xlabel('State')
plt.ylabel('Number of Orders')
plt.xticks(rotation=45)
plt.show()

# Boxplot of avg_order_value by customer_state
sns.boxplot(x='customer_state', y='avg_order_value', data=customer_features)
plt.title('Average Order Value by State')
plt.xlabel('State')
plt.ylabel('Avg Order Value')
plt.xticks(rotation=45)
plt.show()

#### Basket - Analysis

In [ ]:
# Heatmap of basket_matrix correlation
corr_matrix = basket_matrix.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, cmap='coolwarm')
plt.title('Product Category Co-occurrence Heatmap')
plt.show()

# Dendrogram (hierarchical clustering of basket_matrix)
from scipy.cluster.hierarchy import linkage, dendrogram
linkage_matrix = linkage(basket_matrix.T, method='ward')
plt.figure(figsize=(15, 6))
dendrogram(linkage_matrix, labels=basket_matrix.columns, leaf_rotation=90)
plt.title('Product Category Dendrogram (Market Basket Segmentation)')
plt.show()


### Modelling

#### RFM

Recency: Days since the last purchase for each customer.

Frequency: Total number of orders per customer.

Monetary: Total spending by customer.

Scored R, F, M using quantiles: Assigned scores from 1–4 to rank customer behavior.

Created RFM segments: Combined RFM scores to categorize customers (e.g., "444" = best).

Labeled segments: Mapped RFM scores to business-friendly labels like “Loyal” or “At Risk”.

In [ ]:
# Reference date: last purchase date + 1
snapshot_date = orders_full['order_purchase_timestamp'].max() + pd.Timedelta(days=1)

# Aggregate RFM values
rfm = orders_full.groupby('customer_id').agg({
    'order_purchase_timestamp': lambda x: (snapshot_date - x.max()).days,  # Recency
    'order_id': 'count',                                                  # Frequency
    'price': 'sum'                                                        # Monetary
}).reset_index()

rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']

rfm.describe()


In [ ]:
# Score each metric 1–4
rfm['r_score'] = pd.qcut(rfm['recency'], 4, labels=[4, 3, 2, 1]).astype(int)
rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4]).astype(int)
rfm['m_score'] = pd.qcut(rfm['monetary'], 4, labels=[1, 2, 3, 4]).astype(int)

# Create RFM segment
rfm['rfm_segment'] = rfm['r_score'].astype(str) + rfm['f_score'].astype(str) + rfm['m_score'].astype(str)

# Preview
rfm.head()


In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(data=rfm, x='rfm_segment', order=rfm['rfm_segment'].value_counts().index[:15])
plt.title('Top RFM Segments')
plt.xlabel('RFM Segment')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


#### Association Mining

In [ ]:
frequent_itemsets = apriori(basket_matrix, min_support=0.01, use_colnames=True)

rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.1)

# Sort by confidence
rules.sort_values(by='confidence', ascending=False).head(10)

In [ ]:
plt.figure(figsize=(6, 5))
sns.scatterplot(data=rules, x='support', y='confidence',size='lift', palette='coolwarm',hue='lift', sizes=(40, 200), legend='full')

plt.title('Association Rules: Support vs Confidence')
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.legend(title='Lift', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


#### K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
clustering_prepared = imputer.fit_transform(clustering_prepared)
#Try K from 2 to 10
inertia = []
K_range = range(2, 11)

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init='auto')
    km.fit(clustering_prepared)
    inertia.append(km.inertia_)

# Plot
plt.figure(figsize=(6, 4))
plt.plot(K_range, inertia, marker='o')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method for K')
plt.grid(True)
plt.show()


In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42, n_init='auto')
clusters = kmeans.fit_predict(clustering_prepared)

# Add cluster labels to original DataFrame
clustering_data['cluster'] = clusters

# Check cluster distribution
clustering_data['cluster'].value_counts().sort_index()


In [ ]:
# Average values per cluster
cluster_profiles = clustering_data.groupby('cluster').mean(numeric_only=True)
cluster_profiles


In [ ]:
# Normalize cluster profiles
profile_norm = cluster_profiles.copy()
profile_norm = (profile_norm - profile_norm.min()) / (profile_norm.max() - profile_norm.min())

# Prepare for radar
labels = profile_norm.columns.tolist()
num_vars = len(labels)

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

for i in profile_norm.index:
    values = profile_norm.loc[i].tolist()
    values += values[:1]  # loop back to start
    angles = [n / float(num_vars) * 2 * pi for n in range(num_vars)]
    angles += angles[:1]

    ax.plot(angles, values, label=f'Cluster {i}')
    ax.fill(angles, values, alpha=0.1)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels)
plt.title('Cluster Profiles (Radar Chart)')
plt.legend()
plt.show()
